#### <font color='maroon'><b>Amazon - Microwave oven - Web Scraping - using Selenium :

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
import re
import pandas as pd
import numpy as np

In [2]:
# Configure Chrome options to disable images and ads
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--blink-settings=imagesEnabled=false")
chrome_options.add_argument("--dns-prefetch-disable")

In [3]:
# Create a new Chrome driver instance using the managed ChromeDriver
driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.81M/6.81M [00:01<00:00, 5.67MB/s]
C:\Users\Dell\AppData\Local\Temp\ipykernel_4560\3777093032.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)


In [4]:
driver.get("https://www.amazon.in/s?rh=n%3A84514739031&fs=true&ref=lp_84514739031_sar")

In [12]:
product_links = []

In [13]:
def scrape_product_links(search_results):
    for result in search_results:
        try:
            product_link = result.find_element(By.CSS_SELECTOR, "a.a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal").get_attribute("href")
            if "sspa/click?" not in product_link:
                product_links.append(product_link)
        except:
            continue

In [7]:
search_results = driver.find_elements(By.CSS_SELECTOR, "div[data-component-type='s-search-result']")

In [14]:
scrape_product_links(search_results)

In [11]:
len(search_results)

26

In [16]:
len(product_links)

24

In [29]:
next_page_button = driver.find_element(By.CSS_SELECTOR, "a.s-pagination-item.s-pagination-next")

In [30]:
# Check if the next page button is enabled and clickable
if "s-pagination-disabled" not in next_page_button.get_attribute("class"):
    # Click the next page button
    next_page_button.click()
    # Wait for the next page to load
    driver.implicitly_wait(10)
    search_results = driver.find_elements(By.CSS_SELECTOR, "div[data-component-type='s-search-result']")
    driver.implicitly_wait(10)
    scrape_product_links(search_results)

In [31]:
len(product_links)

30

In [33]:
product_links

['https://www.amazon.in/Whirlpool-MAGICOOK-PRO-20SE-BLACK/dp/B07YWYGJYD/ref=sr_1_3?qid=1686464087&s=kitchen&sr=1-3',
 'https://www.amazon.in/Panasonic-Microwave-NN-ST26JMFDG-Silver-Menus/dp/B08CL8XF75/ref=sr_1_4?qid=1686464087&s=kitchen&sr=1-4',
 'https://www.amazon.in/IFB-Microwave-25PM2S-IFBJ0-Silver/dp/B085V1JD27/ref=sr_1_5?qid=1686464087&s=kitchen&sr=1-5',
 'https://www.amazon.in/Samsung-Convection-Microwave-MC28A5013AK-TL/dp/B09XBMXQ34/ref=sr_1_6?qid=1686464087&s=kitchen&sr=1-6',
 'https://www.amazon.in/Samsung-Convection-Microwave-CE1041DSB3-TL/dp/B09VH1Q7CM/ref=sr_1_7?qid=1686464087&s=kitchen&sr=1-7',
 'https://www.amazon.in/Panasonic-Solo-Microwave-NN-SM25JBFDG-Black/dp/B08DYCWC9R/ref=sr_1_8?qid=1686464087&s=kitchen&sr=1-8',
 'https://www.amazon.in/IFB-Litre-Microwave-20PM-MEC2-White/dp/B09HMGHFB6/ref=sr_1_9?qid=1686464087&s=kitchen&sr=1-9',
 'https://www.amazon.in/IFB-Convection-Microwave-30BRC2-Black/dp/B016ZLKJSU/ref=sr_1_10?qid=1686464087&s=kitchen&sr=1-10',
 'https://www.a

There are 30 product links leaving out the sponsored ones.

In [45]:
product_title = []
sales_price = []
mrp = []
discounts = []
ratings = []
no_of_ratings = []
seller_name = []
technical_details = []
additional_info = []

In [46]:
for link in product_links:
    driver.get(link)
    driver.implicitly_wait(10)
    
    # titles
    
    try:
        title = driver.find_element(By.ID, "productTitle").text
        product_title.append(title)
    except:
        product_title.append('NA')
    
    # sales price

    try:
        sl_pr = driver.find_element(By.CSS_SELECTOR, "#newAccordionRow_0 span.a-color-price").text.strip()
        sales_price.append(sl_pr)
    except:
        sales_price.append('NA')
   
    
    # market price
    
    try:
        mk_pr = driver.find_element(By.CSS_SELECTOR, "#newAccordionRow_0 span.a-color-secondary").text.strip()
        mrp.append(mk_pr)
    except:
        mrp.append('NA')
    
    try:
        discount = driver.find_element(By.CSS_SELECTOR, "div#corePriceDisplay_desktop_feature_div.celwidget span").text
        discounts.append(discount)
    except:
        discounts.append('NA')
    
    # product ratings
    
    try:
        rating_element = driver.find_element(By.CSS_SELECTOR, "div#averageCustomerReviews span.a-icon-alt")
        rating = rating_element.get_attribute("innerHTML").split()[0]
        ratings.append(rating)
    except:
        ratings.append('NA')
            
    # number of ratings
    
    try:
        no = driver.find_element(By.CSS_SELECTOR, "#acrCustomerReviewText").text.split()[0]
        no_of_ratings.append(no)
    except:
        no_of_ratings.append('NA')
    
    
   # seller 
    
    try:
        sell = driver.find_element(By.CSS_SELECTOR, "#merchant-info a").text
        seller_name.append(sell)
    except:
        seller_name.append('NA')
            
    # technical details
    
    try:
        tech_details_table = driver.find_element(By.CSS_SELECTOR, "#productDetails_techSpec_section_1")
        rows = tech_details_table.find_elements(By.XPATH, ".//tr")

        details = {}
        for row in rows:
            cells = row.find_elements(By.XPATH, ".//th|td")
            if len(cells) == 2:
                key = cells[0].text.strip()
                value = cells[1].text.strip()
                details[key] = value

        technical_details.append(details)
    except:
        technical_details.append({})
        
    # additional information
        
    try:
        adl_details_table = driver.find_element(By.CSS_SELECTOR, "#productDetails_db_sections")
        rows = adl_details_table.find_elements(By.XPATH, ".//tr")

        details = {}
        for row in rows:
            cells = row.find_elements(By.XPATH, ".//th|td")
            if len(cells) == 2:
                key = cells[0].text.strip()
                value = cells[1].text.strip()
                details[key] = value

        additional_info.append(details)
    except:
        additional_info.append({})
            
        

In [47]:
print('product_title')
print()
print(product_title)
print('-'*50)
print('sales_price')
print()
print(sales_price)
print('-'*50)
print('market retail price')
print()
print(mrp)
print('-'*50)
print('discounts')
print()
print(discounts)
print('-'*50)
print('star_rating')
print()
print(ratings)
print('-'*50)
print('no_of_ratings')
print()
print(no_of_ratings)
print('-'*50)
print('seller_name')
print()
print(seller_name)
print('-'*50)
print('technical_details')
print()
print(technical_details)
print('-'*50)
print('additional_info')
print()
print(additional_info)

product_title

['Whirlpool 20 L Solo Microwave Oven (MAGICOOK PRO 20SE BLACK)', 'Panasonic 20L Solo Microwave Oven (NN-ST26JMFDG, Silver, 51 Auto Menus)', 'IFB 25 L Solo Microwave Oven (25PM2S, Silver)', 'Samsung 28 L Convection Microwave Oven (MC28A5013AK/TL, Black, Curd Anytime)', 'Samsung 28 L Convection Microwave Oven (CE1041DSB3/TL, Black, SlimFry)', 'Panasonic 20L Solo Microwave Oven (NN-SM25JBFDG,Black)', 'IFB 20 Litre Solo Microwave Oven (20PM-MEC2) White', 'IFB 30 L Convection Microwave Oven (30BRC2, Black, With Starter Kit)', 'Bajaj 1701 MT 17L Solo Microwave Oven, White', 'LG 20 L Solo Microwave Oven (MS2043DB, Black)', 'LG 28 L Convection Microwave Oven (MC2846BV, Black)', 'Panasonic 23L Convection Microwave Oven(NN-CT353BFDG,Black Mirror, 360° Heat Wrap, Magic Grill)', 'IFB 20 L Convection Microwave Oven (20SC2, Metallic Silver, With Starter Kit)', 'Morphy Richards 30 MCGR Deluxe 30L Convection Microwave Oven with Motorised Rotisserie, 200 Autocook Menus and Child Lock Fea

#### <font color='maroon'><b>Data Cleaning :

In [48]:
def clean_list_elements(lst):
    cleaned_list = []
    for element in lst:
        cleaned_element = re.sub(r'[,%-]', '', str(element))
        cleaned_list.append(cleaned_element)
    return cleaned_list

In [49]:
discounts_c = clean_list_elements(discounts)
print(discounts_c)

['32', '21', 'NA', '26', '27', '19', '15', '23', '22', 'NA', '29', '31', '26', '44', '14', '26', '25', '8', '29', '15', 'NA', '26', '32', 'NA', 'NA', '28', '6', '3', '47', '34']


In [80]:
mrp_c = clean_list_elements(mrp)

In [81]:
sales_price_c = clean_list_elements(sales_price)

In [53]:
number_of_ratings = clean_list_elements(no_of_ratings)
print(number_of_ratings)

['1564', '4333', '892', '3652', '7751', '2565', '3704', '5308', '5272', '2093', '2574', '1428', '4883', '238', '138', '2820', '499', '202', '2257', '1614', '635', '652', '1670', '2339', '59', '899', '2749', '235', '20', '50']


#### <font color='maroon'><b>Converting the data in pandas dataframe :

In [82]:
df1 = pd.DataFrame({'product_url':product_links,
                  'product_title':product_title,
                  'star_rating of product out of 5':ratings,
                  'number_of_ratings':number_of_ratings,
                  'mrp in Rs/-':mrp_c,
                  'sales_price in Rs/-':sales_price_c,
                  'discount in %':discounts_c,
                  'seller_name':seller_name,
                  'technical_details':technical_details,
                  'additional_information':additional_info})

In [83]:
df1.head()

,product_url,product_title,star_rating of product out of 5,number_of_ratings,mrp in Rs/-,sales_price in Rs/-,discount in %,seller_name,technical_details,additional_information
0,https://www.amazon.in/Whirlpool-MAGICOOK-PRO-2...,Whirlpool 20 L Solo Microwave Oven (MAGICOOK P...,4.1,1564,8755.00,5920.00,32,DAWNTECH ELECTRONICS PRIVATE LIMITED,"{'Brand': 'Whirlpool', 'Model': 'MAGICOOK PRO ...","{'ASIN': 'B07YWYGJYD', 'Customer Reviews': '4...."
1,https://www.amazon.in/Panasonic-Microwave-NN-S...,Panasonic 20L Solo Microwave Oven (NN-ST26JMFD...,4.4,4333,7490.00,5890.00,21,DAWNTECH ELECTRONICS PRIVATE LIMITED,"{'Brand': 'Panasonic', 'Model': 'NN-ST26JMFDG'...","{'ASIN': 'B08CL8XF75', 'Customer Reviews': '4...."
2,https://www.amazon.in/IFB-Microwave-25PM2S-IFB...,"IFB 25 L Solo Microwave Oven (25PM2S, Silver)",4.3,892,NA,NA,NA,NA,"{'Brand': 'IFB', 'Model': '25PM2S', 'Capacity'...","{'ASIN': 'B085V1JD27', 'Customer Reviews': '4...."
3,https://www.amazon.in/Samsung-Convection-Micro...,Samsung 28 L Convection Microwave Oven (MC28A5...,4.3,3652,15590.00,11590.00,26,DAWNTECH ELECTRONICS PRIVATE LIMITED,"{'Brand': 'Samsung', 'Model': 'MC28A5013AK/TL'...","{'ASIN': 'B09XBMXQ34', 'Customer Reviews': '4...."
4,https://www.amazon.in/Samsung-Convection-Micro...,Samsung 28 L Convection Microwave Oven (CE1041...,4.4,7751,16990.00,12390.00,27,DAWNTECH ELECTRONICS PRIVATE LIMITED,"{'Brand': 'Samsung', 'Model': 'CE1041DSB3/TL',...","{'ASIN': 'B09VH1Q7CM', 'Customer Reviews': '4...."


In [87]:
df1.replace('NA', np.nan, inplace=True)

In [88]:
df1['sales_price in Rs/-'] = df1['sales_price in Rs/-'].astype(float).astype('Int64')

In [90]:
df1['mrp in Rs/-'] = df1['mrp in Rs/-'].astype(float).astype('Int64')

In [61]:
# technical details in pandas dataframe

df_tech = pd.DataFrame(technical_details)
df_tech.head()

,Brand,Model,Capacity,Annual Energy Consumption,Installation Type,Part Number,Special Features,Oven Cooking Mode,Burner Type,Colour,...,Manufacturer,Country of Origin,Energy Consumption,Batteries Included,Control Console,Imported By,Volume Capacity Name,Door Material,Form Factor,Certification
0,Whirlpool,MAGICOOK PRO 20SE BLACK,20 Kilograms,150 Watts,Countertop,MAGICOOK PRO 20SE BLACK,Auto_cook,Solo,Ceran / Ceramic,Black,...,Whirlpool of India Ltd,China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Panasonic,NN-ST26JMFDG,20 litres,NaN,Countertop,NN-ST26JMFDG,"Oven Safe, Auto Cook, Defrost, Turntable",Solo,NaN,Silver,...,Panasonic,Thailand,800 Watts,No,NaN,NaN,NaN,NaN,NaN,NaN
2,IFB,25PM2S,25 litres,NaN,Freestanding,25PM2S,"61 Auto-cook menus, Disinfect, Deodorize, Time...",Solo,NaN,Silver,...,Guangdong Galanz Microwave Oven and Electrical...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Samsung,MC28A5013AK/TL,28 litres,NaN,Freestanding,MW5000H,Defrost,"Grill, Microwave, Convection",Ceramic,Black,...,Samsung india Pvt Ltd,Malaysia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Samsung,CE1041DSB3/TL,28 litres,NaN,Freestanding,VG-SCLA00G/XL,Defrost,"Grill, Microwave, Convection",Ceramic,Black,...,Samsung,Malaysia,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
df_tech['Capacity']

0     20 Kilograms
1        20 litres
2        25 litres
3        28 litres
4        28 litres
5        20 litres
6        20 litres
7        30 litres
8        17 litres
9        20 litres
10       28 litres
11       23 litres
12       20 litres
13       30 litres
14       20 litres
15       30 litres
16       20 litres
17       30 litres
18       32 litres
19       32 litres
20       20 litres
21       32 litres
22       28 litres
23       20 litres
24       28 litres
25       21 litres
26       28 litres
27       32 litres
28       28 litres
29       25 litres
Name: Capacity, dtype: object

In [70]:
df_tech['Capacity'] = df_tech['Capacity'].apply(lambda x:re.findall('\d+\.?\d*',x)[0])
df_tech['Capacity']

0     20
1     20
2     25
3     28
4     28
5     20
6     20
7     30
8     17
9     20
10    28
11    23
12    20
13    30
14    20
15    30
16    20
17    30
18    32
19    32
20    20
21    32
22    28
23    20
24    28
25    21
26    28
27    32
28    28
29    25
Name: Capacity, dtype: object

In [71]:
df_tech.rename(columns={'Capacity':'Capacity in litres'},inplace=True)

In [60]:
# additional information in pandas dataframe 

df_add = pd.DataFrame(additional_info)
df_add.head()

,ASIN,Customer Reviews,Best Sellers Rank,Date First Available,Manufacturer,Packer,Importer,Item Weight,Item Dimensions LxWxH,Net Quantity,Generic Name
0,B07YWYGJYD,"4.1\n1,564 ratings\n4.1 out of 5 stars","#1,329 in Home & Kitchen (See Top 100 in Home ...",13 December 2019,"Whirlpool of India Ltd, Whirlpool",Whirlpool,"Whirlpool of India Ltd. Plot No. A-4, MIDC Ran...",2 kg 650 g,177.2 x 103.1 x 137.8 Centimeters,1 set,Microwave Ovens
1,B08CL8XF75,"4.4\n4,333 ratings\n4.4 out of 5 stars","#1,615 in Home & Kitchen (See Top 100 in Home ...",8 July 2020,"Panasonic, Panasonic",Panasonic,Panasonic,10 kg 500 g,44.3 x 34 x 25.8 Centimeters,1 count,Microwave Ovens
2,B085V1JD27,4.3\n892 ratings\n4.3 out of 5 stars,"#2,486 in Home & Kitchen (See Top 100 in Home ...",12 March 2020,Guangdong Galanz Microwave Oven and Electrical...,"IFB Industries Ltd.L-1, Verna, Goa, India-4037...","IFB Industries Ltd.L-1, Verna, Goa, India-4037...",13 kg 400 g,41.5 x 48.3 x 28.1 Centimeters,1 Count,Microwave Ovens
3,B09XBMXQ34,"4.3\n3,652 ratings\n4.3 out of 5 stars","#3,068 in Home & Kitchen (See Top 100 in Home ...",6 April 2022,"Samsung india Pvt Ltd, Email: support.india@sa...",Email: support.india@samsung.com Tel (Toll Fre...,Email: support.india@samsung.com Tel (Toll Fre...,17 kg 500 g,47.5 x 51.7 x 31 Centimeters,1 Case,Convection Microwave Oven
4,B09VH1Q7CM,"4.4\n7,751 ratings\n4.4 out of 5 stars","#5,366 in Home & Kitchen (See Top 100 in Home ...",14 March 2022,"Samsung, Email: support.india@samsung.com Tel ...",Email: support.india@samsung.com Tel (Toll Fre...,Email: support.india@samsung.com Tel (Toll Fre...,17 kg 500 g,47.5 x 51.7 x 31 Centimeters,1 Case,Convection Microwave Oven


In [62]:
# Separating the home kitchen rank and the sub-category rank as separate columns

df_add['Home & Kitchen Rank'] = df_add['Best Sellers Rank'].str.split('\n').str[0].str.split('in Home & Kitchen').str[0].str.replace('#', '').str.replace(',', '').str.strip()
df_add['Microwave Ovens Rank'] = df_add['Best Sellers Rank'].str.split('\n').str[1].str.split('in Microwave Ovens').str[0].str.replace('#', '').str.replace(',', '').str.strip()

In [63]:
# Removing the reduntant columns

df_add = df_add.drop(['Customer Reviews','Best Sellers Rank','Manufacturer'],axis=1)
df_add.head()

,ASIN,Date First Available,Packer,Importer,Item Weight,Item Dimensions LxWxH,Net Quantity,Generic Name,Home & Kitchen Rank,Microwave Ovens Rank
0,B07YWYGJYD,13 December 2019,Whirlpool,"Whirlpool of India Ltd. Plot No. A-4, MIDC Ran...",2 kg 650 g,177.2 x 103.1 x 137.8 Centimeters,1 set,Microwave Ovens,1329,1
1,B08CL8XF75,8 July 2020,Panasonic,Panasonic,10 kg 500 g,44.3 x 34 x 25.8 Centimeters,1 count,Microwave Ovens,1615,2
2,B085V1JD27,12 March 2020,"IFB Industries Ltd.L-1, Verna, Goa, India-4037...","IFB Industries Ltd.L-1, Verna, Goa, India-4037...",13 kg 400 g,41.5 x 48.3 x 28.1 Centimeters,1 Count,Microwave Ovens,2486,4
3,B09XBMXQ34,6 April 2022,Email: support.india@samsung.com Tel (Toll Fre...,Email: support.india@samsung.com Tel (Toll Fre...,17 kg 500 g,47.5 x 51.7 x 31 Centimeters,1 Case,Convection Microwave Oven,3068,6
4,B09VH1Q7CM,14 March 2022,Email: support.india@samsung.com Tel (Toll Fre...,Email: support.india@samsung.com Tel (Toll Fre...,17 kg 500 g,47.5 x 51.7 x 31 Centimeters,1 Case,Convection Microwave Oven,5366,9


In [66]:
pd.set_option('display.max_columns',None)

In [91]:
# Concatenating all details in one dataframe

df = df1.join(df_tech).join(df_add)
df.head()

,product_url,product_title,star_rating of product out of 5,number_of_ratings,mrp in Rs/-,sales_price in Rs/-,discount in %,seller_name,technical_details,additional_information,Brand,Model,Capacity in litres,Annual Energy Consumption,Installation Type,Part Number,Special Features,Oven Cooking Mode,Burner Type,Colour,Voltage,Wattage,Fuel Type,Defrost System,Door Orientation,Material,Included Components,Batteries Required,Manufacturer,Country of Origin,Energy Consumption,Batteries Included,Control Console,Imported By,Volume Capacity Name,Door Material,Form Factor,Certification,ASIN,Date First Available,Packer,Importer,Item Weight,Item Dimensions LxWxH,Net Quantity,Generic Name,Home & Kitchen Rank,Microwave Ovens Rank
0,https://www.amazon.in/Whirlpool-MAGICOOK-PRO-2...,Whirlpool 20 L Solo Microwave Oven (MAGICOOK P...,4.1,1564,8755,5920,32,DAWNTECH ELECTRONICS PRIVATE LIMITED,"{'Brand': 'Whirlpool', 'Model': 'MAGICOOK PRO ...","{'ASIN': 'B07YWYGJYD', 'Customer Reviews': '4....",Whirlpool,MAGICOOK PRO 20SE BLACK,20,150 Watts,Countertop,MAGICOOK PRO 20SE BLACK,Auto_cook,Solo,Ceran / Ceramic,Black,230 Volts,800 Watts,Electric,Defrost,Left,Iron,1,No,Whirlpool of India Ltd,China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B07YWYGJYD,13 December 2019,Whirlpool,"Whirlpool of India Ltd. Plot No. A-4, MIDC Ran...",2 kg 650 g,177.2 x 103.1 x 137.8 Centimeters,1 set,Microwave Ovens,1329,1
1,https://www.amazon.in/Panasonic-Microwave-NN-S...,Panasonic 20L Solo Microwave Oven (NN-ST26JMFD...,4.4,4333,7490,5890,21,DAWNTECH ELECTRONICS PRIVATE LIMITED,"{'Brand': 'Panasonic', 'Model': 'NN-ST26JMFDG'...","{'ASIN': 'B08CL8XF75', 'Customer Reviews': '4....",Panasonic,NN-ST26JMFDG,20,NaN,Countertop,NN-ST26JMFDG,"Oven Safe, Auto Cook, Defrost, Turntable",Solo,NaN,Silver,230 Volts,800 Watts,Electric,Defrost,Left,Iron,"MWO Unit, Turntable, Rotating Ring, User Manual",No,Panasonic,Thailand,800 Watts,No,NaN,NaN,NaN,NaN,NaN,NaN,B08CL8XF75,8 July 2020,Panasonic,Panasonic,10 kg 500 g,44.3 x 34 x 25.8 Centimeters,1 count,Microwave Ovens,1615,2
2,https://www.amazon.in/IFB-Microwave-25PM2S-IFB...,"IFB 25 L Solo Microwave Oven (25PM2S, Silver)",4.3,892,<NA>,<NA>,NaN,NaN,"{'Brand': 'IFB', 'Model': '25PM2S', 'Capacity'...","{'ASIN': 'B085V1JD27', 'Customer Reviews': '4....",IFB,25PM2S,25,NaN,Freestanding,25PM2S,"61 Auto-cook menus, Disinfect, Deodorize, Time...",Solo,NaN,Silver,230 Volts,50,Electric,Defrost,Left,Stainless Steel,"1 Microwave Oven, Warranty Card and User Manual",No,Guangdong Galanz Microwave Oven and Electrical...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B085V1JD27,12 March 2020,"IFB Industries Ltd.L-1, Verna, Goa, India-4037...","IFB Industries Ltd.L-1, Verna, Goa, India-4037...",13 kg 400 g,41.5 x 48.3 x 28.1 Centimeters,1 Count,Microwave Ovens,2486,4
3,https://www.amazon.in/Samsung-Convection-Micro...,Samsung 28 L Convection Microwave Oven (MC28A5...,4.3,3652,15590,11590,26,DAWNTECH ELECTRONICS PRIVATE LIMITED,"{'Brand': 'Samsung', 'Model': 'MC28A5013AK/TL'...","{'ASIN': 'B09XBMXQ34', 'Customer Reviews': '4....",Samsung,MC28A5013AK/TL,28,NaN,Freestanding,MW5000H,Defrost,"Grill, Microwave, Convection",Ceramic,Black,230 Volts,900 Watts,Electric,NaN,Left,Body : Stainless Steel,"Glass Turntable, Wire Rack, Quick Guide Label",No,Samsung india Pvt Ltd,Malaysia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B09XBMXQ34,6 April 2022,Email: support.india@samsung.com Tel (Toll Fre...,Email: support.india@samsung.com Tel (Toll Fre...,17 kg 500 g,47.5 x 51.7 x 31 Centimeters,1 Case,Convection Microwave Oven,3068,6
4,https://www.amazon.in/Samsung-Convection-Micro...,Samsung 28 L Convection Microwave Oven (CE1041...,4.4,7751,16990,12390,27,DAWNTECH ELECTRONICS PRIVATE LIMITED,"{'Brand': 'Samsung', 'Model': 'CE1041DSB3/TL',...","{'ASIN': 'B09VH1Q7CM', 'Customer Reviews': '4....",Samsung,CE1041DSB3/TL,28,NaN,Freestanding,VG-SCLA00G/XL,Defrost,"Grill, Microwave, Convection",Ceramic,Black,230 Volts,900 Watts,Electric,NaN,Left,"Cavity : Ceramic Enamel Cavity, Door : Glass, ...","Glass Turntable, Wi

In [65]:
df.shape

(30, 48)

In [92]:
df.to_csv('Amazon_microwave_oven.csv',index=False)